#데이터 로드 및 전처리
3축 가속도계(accelerometer)와 3축 자이로스코프(gyroscope)를 활용해 측정된 센서 데이터

- train_features.csv (1875000, 8)

> id 별 600 time 간 동작 데이터

>id 3125개 x 600 time =1875000 데이터


- train_labels.csv (3125, 3)

>id 별 동작과 동작 label(61개)

- test_features.csv (469200, 8)

>id 별 600 time간 동작 데이터

>id 782개 x 600 time =469200 데이터

-sample_submission.csv (782, 62)

>id별 동작을 예측해 작성하는 csv

In [ ]:
import math
import time
from itertools import chain
import numpy as np
import pandas as pd
import torch 
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, TensorDataset
from tqdm.auto import tqdm
from torch.optim import Optimizer
from torch.optim.lr_scheduler import LambdaLR

In [ ]:
train = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/train_features.csv")
train_labels = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/train_labels.csv")
test=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/test_features.csv')
submission=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/sample_submission.csv')
train

,id,time,acc_x,acc_y,acc_z,gy_x,gy_y,gy_z
0,0,0,1.206087,-0.179371,-0.148447,-0.591608,-30.549010,-31.676112
1,0,1,1.287696,-0.198974,-0.182444,0.303100,-39.139103,-24.927216
2,0,2,1.304609,-0.195114,-0.253382,-3.617278,-44.122565,-25.019629
3,0,3,1.293095,-0.230366,-0.215210,2.712986,-53.597843,-27.454013
4,0,4,1.300887,-0.187757,-0.222523,4.286707,-57.906561,-27.961234
...,...,...,...,...,...,...,...,...
1874995,3124,595,-0.712530,-0.658357,0.293707,-29.367857,-104.013664,-76.290437
1874996,3124,596,-0.683037,-0.658466,0.329223,-30.149089,-101.796809,-76.625087
1874997,3124,597,-0.664730,-0.666625,0.364114,-27.873095,-98.776072,-79.365125
1874998,3124,598,-0.630534,-0.682565,0.373696,-23.636550,-99.139495,-80.259478


In [ ]:
#id 별로 각각 600개의 데이터가 있음을 확인   
train['id'].value_counts()

2047    600
1347    600
2378    600
1354    600
2379    600
       ... 
1699    600
675     600
2722    600
1698    600
0       600
Name: id, Length: 3125, dtype: int64

In [ ]:
train = train.iloc[:,2:]
train

,acc_x,acc_y,acc_z,gy_x,gy_y,gy_z
0,1.206087,-0.179371,-0.148447,-0.591608,-30.549010,-31.676112
1,1.287696,-0.198974,-0.182444,0.303100,-39.139103,-24.927216
2,1.304609,-0.195114,-0.253382,-3.617278,-44.122565,-25.019629
3,1.293095,-0.230366,-0.215210,2.712986,-53.597843,-27.454013
4,1.300887,-0.187757,-0.222523,4.286707,-57.906561,-27.961234
...,...,...,...,...,...,...
1874995,-0.712530,-0.658357,0.293707,-29.367857,-104.013664,-76.290437
1874996,-0.683037,-0.658466,0.329223,-30.149089,-101.796809,-76.625087
1874997,-0.664730,-0.666625,0.364114,-27.873095,-98.776072,-79.365125
1874998,-0.630534,-0.682565,0.373696,-23.636550,-99.139495,-80.259478


In [ ]:
test = test.iloc[:,2:]
test

,acc_x,acc_y,acc_z,gy_x,gy_y,gy_z
0,-0.628100,-0.160155,0.151487,49.665357,88.435961,13.597668
1,-0.462548,0.012462,-0.053726,56.953059,96.185341,16.278458
2,-0.363481,-0.091789,-0.130004,29.557396,93.836453,13.329043
3,-0.351750,-0.239870,-0.193053,23.686172,88.608721,13.449771
4,-0.312934,-0.123762,-0.318621,20.410071,85.327707,13.884912
...,...,...,...,...,...,...
469195,0.104191,-0.784979,0.639513,-10.475346,14.095361,-190.358982
469196,0.103297,-0.758954,0.615687,-25.360272,-8.523018,-180.393291
469197,0.128294,-0.749389,0.586184,-27.917723,-23.186245,-162.624160
469198,0.104130,-0.692731,0.573397,-27.847980,-30.407555,-138.761676


In [ ]:
#0~599개
def sliding_window():
   time_dictionary_window=[]
   window = []
   start_point = 0
   end_point = 1875000
   for cursor in tqdm(range(start_point, end_point, 3)):
        if (cursor+3) % 600 == 0 :
          continue
        data = np.array(train.iloc[cursor : cursor+6])
        time_dictionary_window.append(data)
        #window.append(data)

        
   return time_dictionary_window

In [ ]:
#0~599개
def sliding_window_test():
   time_dictionary_window=[]
   window = []
   start_point = 0
   end_point = 469200
   for cursor in tqdm(range(start_point, end_point, 3)):
        if (cursor+3) % 600 == 0 :
          continue
        data = np.array(test.iloc[cursor : cursor+6])
        time_dictionary_window.append(data)
        #window.append(data)

        
   return time_dictionary_window

In [ ]:
time_window = sliding_window()
time_array = np.array(time_window)
time_array.shape

(621875, 6, 6)

In [ ]:
time_window_test = sliding_window_test()
time_array_test = np.array(time_window_test)
time_array_test.shape

(155618, 6, 6)

In [ ]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, LSTM

In [ ]:
X=tf.reshape(time_array,[-1, 199, 36])
X.shape

TensorShape([3125, 199, 36])

In [ ]:
time_array.shape

(621875, 6, 6)

In [ ]:
y = tf.keras.utils.to_categorical(train_labels['label']) 
y.shape

(3125, 61)

In [ ]:
#가벼운 모델 생성
model = Sequential()
model.add(LSTM(32, input_shape=(199, 36)))
model.add(Dense(128, activation='relu'))
model.add(Dense(61, activation='softmax'))

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(X, y, epochs=30, batch_size=128, validation_split=0.2)

Epoch 1/30
20/20 [==============================] - 5s 146ms/step - loss: 3.9298 - accuracy: 0.1907 - val_loss: 3.2582 - val_accuracy: 0.4976
Epoch 2/30
20/20 [==============================] - 2s 119ms/step - loss: 3.0941 - accuracy: 0.4824 - val_loss: 2.7745 - val_accuracy: 0.4976
Epoch 3/30
20/20 [==============================] - 2s 118ms/step - loss: 2.7374 - accuracy: 0.4853 - val_loss: 2.6962 - val_accuracy: 0.4976
Epoch 4/30
20/20 [==============================] - 2s 122ms/step - loss: 2.6821 - accuracy: 0.4747 - val_loss: 2.6457 - val_accuracy: 0.4976
Epoch 5/30
20/20 [==============================] - 2s 122ms/step - loss: 2.5706 - accuracy: 0.4870 - val_loss: 2.6148 - val_accuracy: 0.5088
Epoch 6/30
20/20 [==============================] - 2s 121ms/step - loss: 2.5006 - accuracy: 0.4947 - val_loss: 2.5796 - val_accuracy: 0.5120
Epoch 7/30
20/20 [==============================] - 2s 119ms/step - loss: 2.4659 - accuracy: 0.4954 - val_loss: 2.5524 - val_accuracy: 0.5120
Epoch 

In [ ]:
test_X=tf.reshape(time_array_test,[-1, 199,  36])
test_X.shape

TensorShape([782, 199, 36])

In [ ]:
prediction=model.predict(test_X)

In [ ]:
prediction.shape

(782, 61)

In [ ]:
submission

,id,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60
0,3125,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,3126,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,3127,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,3128,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,3129,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
777,3902,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
778,3903,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
779,3904,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
780,3905,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
submission.iloc[:,1:]=prediction

In [ ]:
submission

,id,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60
0,3125,0.003950,0.005083,0.000577,0.001478,0.016668,0.014357,0.023431,0.021551,0.000517,0.012680,0.002919,0.003361,0.006878,0.004324,0.005992,0.005611,0.010885,0.043255,0.008940,0.001366,0.003833,0.010599,0.005824,0.034714,0.007056,0.010516,0.480378,0.002113,0.000450,0.000810,0.010323,0.006499,0.003525,0.014897,0.012823,0.006936,0.010681,0.014702,0.002531,0.000766,0.011917,0.001825,0.006811,0.022563,0.010312,0.004285,0.008374,0.008469,0.012674,0.007511,0.011744,0.006306,0.003927,2.251529e-04,0.002915,0.002951,0.002584,0.006777,0.021447,0.002518,0.010068
1,3126,0.000090,0.002829,0.000003,0.000003,0.000317,0.000230,0.000330,0.003404,0.000012,0.000402,0.000311,0.000537,0.000021,0.001224,0.002624,0.000333,0.000058,0.000057,0.000097,0.000007,0.000093,0.000284,0.002050,0.003202,0.000780,0.000372,0.949494,0.000017,0.000004,0.000003,0.000087,0.000098,0.000072,0.001901,0.000218,0.000106,0.000067,0.000339,0.000025,0.000002,0.003512,0.000043,0.000200,0.001165,0.001979,0.001823,0.000118,0.000080,0.000981,0.001046,0.007219,0.000161,0.000877,9.979036e-07,0.000035,0.000005,0.000043,0.004736,0.000552,0.003080,0.000240
2,3127,0.006204,0.000705,0.000555,0.000260,0.001089,0.013473,0.013047,0.001451,0.003056,0.007806,0.005483,0.000600,0.000297,0.000049,0.000300,0.006784,0.000364,0.000288,0.000610,0.005665,0.003398,0.004841,0.006450,0.002054,0.006665,0.009009,0.773211,0.001307,0.002058,0.000303,0.003156,0.000868,0.001289,0.001851,0.002406,0.007930,0.002017,0.004938,0.007966,0.000612,0.002123,0.000698,0.001500,0.011095,0.001316,0.003475,0.000209,0.003202,0.025711,0.010366,0.007317,0.011715,0.000620,5.001642e-04,0.000733,0.000875,0.001818,0.000498,0.001634,0.000496,0.003683
3,3128,0.000795,0.019144,0.000036,0.000043,0.001928,0.000687,0.000863,0.023389,0.002693,0.011851,0.004618,0.020612,0.000802,0.002855,0.037141,0.001699,0.000309,0.000629,0.000304,0.000008,0.000034,0.001208,0.005548,0.004773,0.010882,0.000614,0.715348,0.000108,0.000183,0.000007,0.001307,0.000848,0.002930,0.002641,0.001278,0.000702,0.000790,0.001264,0.000635,0.000084,0.011758,0.000267,0.000265,0.002988,0.004842,0.008664,0.000222,0.001264,0.007295,0.006735,0.011073,0.004783,0.005795,8.368808e-06,0.000271,0.000063,0.000065,0.018501,0.007227,0.018128,0.008193
4,3129,0.000326,0.002503,0.000079,0.000126,0.002664,0.000435,0.000817,0.001816,0.000039,0.000679,0.003544,0.003312,0.000003,0.000035,0.000493,0.001948,0.000302,0.000704,0.007921,0.000065,0.000036,0.000882,0.000803,0.001674,0.002679,0.000902,0.912538,0.000102,0.000005,0.000026,0.004574,0.001328,0.000089,0.003582,0.000742,0.000081,0.000188,0.007422,0.000037,0.000030,0.002992,0.000096,0.001236,0.000713,0.000107,0.002548,0.003537,0.001562,0.000352,0.002220,0.005070,0.000231,0.000069,1.107548e-05,0.000339,0.000132,0.000057,0.007453,0.000109,0.005576,0.000059
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
777,3902,0.000823,0.006005,0.000074,0.000128,0.001827,0.008822,0.005592,0.004020,0.000661,0.004293,0.004637,0.005749,0.000531,0.001195,0.006071,0.003345,0.000847,0.000349,0.000652,0.000274,0.001641,0.006137,0.017048,0.008783,0.003617,0.007176,0.693455,0.000291,0.000682,0.000663,0.004147,0.004173,0.003372,0.004404,0.001038,0.001597,0.002481,0.002196,0.006433,0.000542,0.024851,0.001730,0.007227,0.013111,0.011355,0.006242,0.003358,0.005592,0.006056,0.004052,0.029987,0.004864,0.008273,1.604826e-04,0.000761,0.000678,0.005892,0.015488,0.000705,0.019071,0.004778
778,3903,0.001500,0.017525,0.000063,0.000136,0.010290,0.003775,0.004719,0.010900,0.000494,0.006300,0.005100,0.008753,0.001853,0.000485,0.064911,0.020785,0.001994,0.006589,0.015805,0.000160,0.000274,0.021228,0.

In [ ]:
submission.to_csv('submission.csv', index=False)